### Training Clock Validation Models

This file mainly prepars models which can identify regoins in a clock dataset where a fail state occured in time of TempCo testing.

#### Imports

In [1]:
# PyTorch
import torch, os
import torch.optim as optim
from torch.utils.data import DataLoader


# Custom functions and classes
from dataHandler import ClockDataset
from classifiers import PtClassifier_V2

device = "cuda" if torch.cuda.is_available() else "cpu"

#### Prepare Clock Dataset

TempCo data: clock data collected during clock testing undergoing temperature cycling over the course of a set period of time.

In [2]:
# Paths containing testing and training datasets
TEST_DATASET  = os.path.join(os.getcwd(), r"data\test")
TRAIN_DATASET = os.path.join(os.getcwd(), r"data\train")

# Creating Datasets
clock_dataset_test = ClockDataset(f_data=TEST_DATASET)
clock_dataset_train = ClockDataset(f_data=TRAIN_DATASET)

# Creating Dataloaders
train_dataloader = DataLoader(
    dataset=clock_dataset_train,
    batch_size=1,
    shuffle=False)

test_dataloader = DataLoader(
    dataset=clock_dataset_test,
    batch_size=1)

print(f"Number of clocks in train dataset: {clock_dataset_train.__len__()}")
print(f"Number of clocks in test dataset: {clock_dataset_test.__len__()}")

Number of clocks in train dataset: 117
Number of clocks in test dataset: 18


In [3]:
# Sample data
sample_param, sample_labels, sample_columns, sample_validation = next(iter(train_dataloader))

print(f"Sample Parameter row:\n{sample_param[0][0]}")
print(f"Sample Label: {sample_labels[0]}")
print(f"Sample Columns: {sample_columns}")
print(f"Sample Validation: {sample_validation[0]}")

print(f"Length of sample parameters: {len(sample_param[0])}")

Sample Parameter row:
tensor([7.3140e-01, 3.2896e+04, 1.3569e+04, 1.1119e+04, 2.4500e+03, 1.3412e+00,
        2.7853e+04, 3.2912e+04, 3.3250e+01, 8.6338e+01, 8.4200e+01, 3.9745e+04,
        3.9508e+04, 2.6864e+04, 5.1200e+02])
Sample Label: tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])
Sample Columns: [('DCL',), ('PDADC',), ('CPTP',), ('CPTF',), ('CPTD',), ('VBC',), ('VFA',), ('XFA',), ('TPCB',), ('VTEMP',), ('CTEMP',), ('VHTR',), ('CHTR',), ('RFL',), ('PDSU',), ('FAIL',)]
Sample Validation: fail
Length of sample parameters: 229364


#### Import Model

In [4]:
NUM_PARAMS  = clock_dataset_train[0][0][0].__len__()
NUM_CLASSES = clock_dataset_train[0][1][0].__len__()
model = PtClassifier_V2(parameters=NUM_PARAMS, classes=NUM_CLASSES)
model = model.to(device)
print(f"Number of parameters for model input: {NUM_PARAMS}")
print(f"Number of classes: {NUM_CLASSES}")

Number of parameters for model input: 15
Number of classes: 1


#### Optimizer and Loss Function

In [5]:
optimizer = optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

#### Training
Note:
    Models take a long time to train ~6 min per clock.
    This is because we are trainning the models on each second of data per clockk.
    Training time can be improved by reducing the datasets by factoring out:
        - Data where clock is idle 
        - Introduce a batch parameter when training per clock to train on batches of seconds   

In [ ]:
from engine import train
NUM_EPOCHS = 1

results = train(
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    model=model,
    loss_fn=criterion,
    epochs=NUM_EPOCHS,
    device=device)

print(results)

''' 
    Implement batch size per training and testing step per clock to reduce training time.
'''

Testing Clock: 0 | fail
y_pred: 1 | Act: 0 | On row number: 23498
Testing Clock: 1 | fail
y_pred: 1 | Act: 0 | On row number: 162335
Testing Clock: 2 | fail
y_pred: 1 | Act: 0 | On row number: 157831
Testing Clock: 3 | fail
y_pred: 1 | Act: 1 | On row number: 233850
Testing Clock: 4 | fail_passedLater
y_pred: 1 | Act: 0 | On row number: 152136
Testing Clock: 5 | fail_passedLater
y_pred: 1 | Act: 0 | On row number: 165931
Testing Clock: 6 | fail_passedLater
y_pred: 1 | Act: 0 | On row number: 144081
Testing Clock: 7 | fail_passedLater
y_pred: 1 | Act: 1 | On row number: 170218
Testing Clock: 8 | pass
y_pred: 1 | Act: 0 | On row number: 165931
Testing Clock: 9 | pass
y_pred: 1 | Act: 0 | On row number: 25676
Testing Clock: 10 | pass
y_pred: 1 | Act: 0 | On row number: 234972
Testing Clock: 11 | pass
y_pred: 1 | Act: 0 | On row number: 158743
Testing Clock: 12 | pass
y_pred: 1 | Act: 0 | On row number: 152470
Testing Clock: 13 | pass
y_pred: 1 | Act: 0 | On row number: 23498
Testing Clock